In [9]:
# Imports
import pandas as pd
import numpy as np
import yfinance as yf
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from statsmodels.tsa.arima.model import ARIMA

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Config
selected_tickers = ['ABBV', 'ADBE', 'ADI', 'ABNB', 'ZTS', 'XOM', 'JNJ', 'ACN', 'CMCSA', 'LMT', 'MSFT', 'AAPL', 'GOOGL', 'AMZN', 'TSLA']
SEQ_LEN = 30
FEATURE_COLUMNS = ['Close', 'Return', 'MA7', 'MA14', 'MA30', 'RSI', 'MACD', 'Signal_Line', 'Rolling_Volatility']

In [10]:
# Helper functions
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


def compute_macd(series, short_window=12, long_window=26, signal_window=9):
    short_ema = series.ewm(span=short_window, adjust=False).mean()
    long_ema = series.ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal


class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        return (1 - pt) ** self.gamma * ce_loss

In [ ]:
# Data download and preprocessing
raw_data = yf.download(selected_tickers, period='3y', interval='1d')['Close']

feature_data = {}
scaler = StandardScaler()

for ticker in selected_tickers:
    df = pd.DataFrame({'Close': raw_data[ticker]})
    df['Return'] = df['Close'].pct_change()
    df['MA7'] = df['Close'].rolling(window=7).mean()
    df['MA14'] = df['Close'].rolling(window=14).mean()
    df['MA30'] = df['Close'].rolling(window=30).mean()
    df['RSI'] = compute_rsi(df['Close'])
    df['MACD'], df['Signal_Line'] = compute_macd(df['Close'])
    df['Rolling_Volatility'] = df['Return'].rolling(window=30).std()
    df['Label_Next_Day'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df['Label_Next_Week'] = (df['Close'].shift(-5) > df['Close']).astype(int)
    df['Label_Next_Month'] = (df['Close'].shift(-20) > df['Close']).astype(int)
    df = df.dropna()
    df[FEATURE_COLUMNS] = scaler.fit_transform(df[FEATURE_COLUMNS])
    feature_data[ticker] = df

[*********************100%***********************]  15 of 15 completed
<ipython-input-11-8ad1c1af6892>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[FEATURE_COLUMNS] = scaler.fit_transform(df[FEATURE_COLUMNS])


In [12]:
# Transformer Model
class StockTransformerMultiTask(nn.Module):
    def __init__(self, feature_size, hidden_dim=128, num_classes=2, nhead=4, num_layers=4):
        super().__init__()
        self.embedding = nn.Linear(feature_size, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, batch_first=True, dropout=0.2)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_day = nn.Linear(hidden_dim, num_classes)
        self.fc_week = nn.Linear(hidden_dim, num_classes)
        self.fc_month = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.fc_day(x), self.fc_week(x), self.fc_month(x)

# Prepare Dataset
X_all, y_all_day, y_all_week, y_all_month = [], [], [], []

for ticker, df in feature_data.items():
    for i in range(SEQ_LEN, len(df) - 20):
        seq_x = df.iloc[i-SEQ_LEN:i][FEATURE_COLUMNS].values
        X_all.append(seq_x)
        y_all_day.append(df.iloc[i]['Label_Next_Day'])
        y_all_week.append(df.iloc[i]['Label_Next_Week'])
        y_all_month.append(df.iloc[i]['Label_Next_Month'])

X_all = np.array(X_all)
y_all_day = np.array(y_all_day)
y_all_week = np.array(y_all_week)
y_all_month = np.array(y_all_month)

X_temp, X_test, y_temp_day, y_test_day, y_temp_week, y_test_week, y_temp_month, y_test_month = train_test_split(
    X_all, y_all_day, y_all_week, y_all_month, test_size=0.1, shuffle=True, random_state=42)

X_train, X_val, y_train_day, y_val_day, y_train_week, y_val_week, y_train_month, y_val_month = train_test_split(
    X_temp, y_temp_day, y_temp_week, y_temp_month, test_size=0.1111, shuffle=True, random_state=42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_day = torch.tensor(y_train_day, dtype=torch.long).to(device)
y_val_day = torch.tensor(y_val_day, dtype=torch.long).to(device)
y_test_day = torch.tensor(y_test_day, dtype=torch.long).to(device)
y_train_week = torch.tensor(y_train_week, dtype=torch.long).to(device)
y_val_week = torch.tensor(y_val_week, dtype=torch.long).to(device)
y_test_week = torch.tensor(y_test_week, dtype=torch.long).to(device)
y_train_month = torch.tensor(y_train_month, dtype=torch.long).to(device)
y_val_month = torch.tensor(y_val_month, dtype=torch.long).to(device)
y_test_month = torch.tensor(y_test_month, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train, y_train_day, y_train_week, y_train_month)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train Transformer
model = StockTransformerMultiTask(feature_size=X_train.shape[2]).to(device)
criterion = FocalLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

EPOCHS = 200
best_val_loss = np.inf
patience = 10
trigger_times = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for data, targets_day, targets_week, targets_month in train_loader:
        optimizer.zero_grad()
        out_day, out_week, out_month = model(data)
        loss = criterion(out_day, targets_day) + criterion(out_week, targets_week) + criterion(out_month, targets_month)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    with torch.no_grad():
        val_day, val_week, val_month = model(X_val)
        val_loss = criterion(val_day, y_val_day) + criterion(val_week, y_val_week) + criterion(val_month, y_val_month)

    scheduler.step(val_loss)
    avg_train_loss = running_loss / len(train_loader)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        best_model_state = model.state_dict()
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    if (epoch+1) % 10 == 0:
        current_lr = scheduler.optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1} - Train Loss: {avg_train_loss:.4f} - Val Loss: {val_loss:.4f} - LR: {current_lr:.6f}")

model.load_state_dict(best_model_state)

# Evaluation
def evaluate(preds, targets):
    preds = preds.argmax(dim=1).cpu().numpy()
    targets = targets.cpu().numpy()
    return accuracy_score(targets, preds), precision_score(targets, preds, zero_division=0), recall_score(targets, preds, zero_division=0), f1_score(targets, preds, zero_division=0)

model.eval()
with torch.no_grad():
    preds_day, preds_week, preds_month = model(X_test)

metrics_day = evaluate(preds_day, y_test_day)
metrics_week = evaluate(preds_week, y_test_week)
metrics_month = evaluate(preds_month, y_test_month)

print("\n=== Transformer Final Test Results ===")
print(f"Next Day   - Acc: {metrics_day[0]:.4f}, Prec: {metrics_day[1]:.4f}, Rec: {metrics_day[2]:.4f}, F1: {metrics_day[3]:.4f}")
print(f"Next Week  - Acc: {metrics_week[0]:.4f}, Prec: {metrics_week[1]:.4f}, Rec: {metrics_week[2]:.4f}, F1: {metrics_week[3]:.4f}")
print(f"Next Month - Acc: {metrics_month[0]:.4f}, Prec: {metrics_month[1]:.4f}, Rec: {metrics_month[2]:.4f}, F1: {metrics_month[3]:.4f}")

Epoch 10 - Train Loss: 0.3983 - Val Loss: 0.4001 - LR: 0.000100
Epoch 20 - Train Loss: 0.3110 - Val Loss: 0.3669 - LR: 0.000100
Epoch 30 - Train Loss: 0.2614 - Val Loss: 0.3413 - LR: 0.000100
Early stopping at epoch 36

=== Transformer Final Test Results ===
Next Day   - Acc: 0.5775, Prec: 0.6060, Rec: 0.5529, F1: 0.5782
Next Week  - Acc: 0.7310, Prec: 0.7749, Rec: 0.7067, F1: 0.7393
Next Month - Acc: 0.8592, Prec: 0.8673, Rec: 0.8698, F1: 0.8685


In [13]:
# ARIMA Baseline
series = feature_data['ABBV']['Close']
train_series = series[:-30]
test_series = series[-30:]

model_arima = ARIMA(train_series, order=(5,1,0)).fit()
forecast = model_arima.forecast(steps=len(test_series))

predicted_direction = (forecast.values > train_series.iloc[-1]).astype(int)
true_direction = (test_series.values > train_series.iloc[-1]).astype(int)

acc_arima = accuracy_score(true_direction, predicted_direction)
prec_arima = precision_score(true_direction, predicted_direction, zero_division=0)
rec_arima = recall_score(true_direction, predicted_direction, zero_division=0)
f1_arima = f1_score(true_direction, predicted_direction, zero_division=0)

print("\n=== ARIMA Baseline (ABBV only) ===")
print(f"Accuracy: {acc_arima:.4f} | Precision: {prec_arima:.4f} | Recall: {rec_arima:.4f} | F1: {f1_arima:.4f}")

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)



=== ARIMA Baseline (ABBV only) ===
Accuracy: 0.8667 | Precision: 0.0000 | Recall: 0.0000 | F1: 0.0000


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [14]:
# Naive Baseline
true_direction = (series.shift(-1) > series).astype(int).dropna()
naive_prediction = (series.shift(0) > series.shift(1)).astype(int).dropna()

acc_naive = accuracy_score(true_direction, naive_prediction)
prec_naive = precision_score(true_direction, naive_prediction, zero_division=0)
rec_naive = recall_score(true_direction, naive_prediction, zero_division=0)
f1_naive = f1_score(true_direction, naive_prediction, zero_division=0)

print("\n=== Naive Baseline (ABBV only) ===")
print(f"Accuracy: {acc_naive:.4f} | Precision: {prec_naive:.4f} | Recall: {rec_naive:.4f} | F1: {f1_naive:.4f}")


=== Naive Baseline (ABBV only) ===
Accuracy: 0.5329 | Precision: 0.5669 | Recall: 0.5669 | F1: 0.5669
